In [1]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict
import os
from typing import List, Dict, Any, Tuple
pd.options.display.float_format = "{:,.2f}".format
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
%cd data/props/
%load_ext autoreload
%autoreload 2
print(os.getcwd)
from properties import drd2
from properties import qed
from properties import penalized_logp
from properties import sas

#print(qed("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"), penalized_logp("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"), sas("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"))
print(qed("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"), penalized_logp("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"), sas("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"), drd2("O=[n+]1c2c(n(O)cc3c4cc(O)ccc1c3c4)CCCCC2"))

/Users/van/Developer/workspace/gellmo/GeLLMO/data/props
<built-in function getcwd>
0.4334298211192694 -10.300947630755612 5.122530176377287 0.0740022497482625


/opt/homebrew/Caskroom/miniforge/base/envs/sk21/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SVC from version 0.18.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)

def save_json(file, data):
    with open(file, 'w') as f:
        json.dump(data, f, indent=2)

In [4]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
# taken from https://github.com/ziqi92/Modof/blob/main/model/properties.py#L34
def pair_similarity(amol, bmol, sim_type):
    if amol is None or bmol is None: 
        return 0.0

    if isinstance(amol, str):
        amol = Chem.MolFromSmiles(amol)
    if isinstance(bmol, str):
        bmol = Chem.MolFromSmiles(bmol)
    if amol is None or bmol is None:
        return 0.0

    if sim_type == "binary":
        fp1 = AllChem.GetMorganFingerprintAsBitVect(amol, 2, nBits=2048, useChirality=False)
        fp2 = AllChem.GetMorganFingerprintAsBitVect(bmol, 2, nBits=2048, useChirality=False)
    else:
        fp1 = AllChem.GetMorganFingerprint(amol, 2, useChirality=False)
        fp2 = AllChem.GetMorganFingerprint(bmol, 2, useChirality=False)

    sim = DataStructs.TanimotoSimilarity(fp1, fp2)
    
    return sim

In [5]:
def compute_FP_sim(smiles):
    # assume there might be invalid smiles
    mols = []
    for smi in smiles:
        mol = Chem.MolFromSmiles(smi)
        if mol:
            mols.append(mol)
            
    sim = np.zeros((len(mols), len(mols)))
    for i in range(len(mols)):
        for j in range(i, len(mols)):
            sim[i, j] = sim[j, i] = pair_similarity(mols[i], mols[j], "binary")
    return sim

In [6]:
def normalize_prop(val, min_val, max_val):
    normalized_val = (val - min_val) / (max_val - min_val)
    return max(0, min(1, normalized_val))

In [7]:
def find_best_optimized(input_prop, output_prop, property):
    """
    Find the best optimized SMILES that satisfies constraints and maximizes improvement.
    Args:
        input_prop (dict): Properties of the input molecule.
        output_prop (dict): Properties of the optimized molecules.
        property (list): List of properties to optimize.
    
    Returns:
        int: Index of the best optimized molecule.
    """
    best_idx = None
    best_improvement = -1
    num_candidates = len(output_prop[property[0]])

    for i in range(num_candidates):
        improvement = 0
        # Skip candidate if the change in property is in the wrong direction
        wrong_direction = False
        for prop in property:
            input_val = input_prop[prop]
            output_val = output_prop[prop][i]
            #print(i, prop, input_val, output_val)
            if prop == "mutagenicity":
                if output_val >= input_val:  # Mutagenicity must decrease
                    wrong_direction = True
                    break
            else:
                if output_val <= input_val:  # Other properties must increase
                    wrong_direction = True
                    break

        #print(wrong_direction)
        if wrong_direction:
            continue  # Skip this candidate

        for prop in property:
            input_val = input_prop[prop]
            output_val = output_prop[prop][i]
    
            if input_val == 0:
                improvement += output_val
            else:
                improvement += (output_val - input_val) / abs(input_val) if prop != 'mutagenicity' else (input_val - output_val) / abs(input_val)
            #print(i, prop, input_prop[prop], output_prop[prop][i], improvement)
        
        if improvement >= best_improvement:
            best_improvement = improvement
            best_idx = i
            
    #print(f"Best improvement: {best_improvement}, index: {best_idx}")
    return best_idx

In [8]:
best_idx = find_best_optimized(
    input_prop={"qed": 0.5, "bbbp": 0.6, "drd2":0, "plogp": -3.0, "mutagenicity": 0.9},
    output_prop={
        "qed": [0.8, 0.7],
        "bbbp": [0.7, 0.7],
        "drd2": [0.1, 0.1],
        "plogp": [2.0, 2.0],
        "mutagenicity": [0.71, 0.6]
    },
    property=["qed", "bbbp", "drd2", "plogp", "mutagenicity"]
)
print("Best index:", best_idx)

Best index: 0


### Processing properties of generated molecules

In [9]:
def compute_props(output_smiles_path, admet_props_path, props_path):
    """computes the logp,qed,drd2,sas for the optimized smiles"""
    output_smiles = []
    with open(output_smiles_path, 'r') as f:
        for line in f:
            output_smiles.extend(line.strip().split(','))
    
    output_smiles = set(output_smiles)
    # read the existing admet properties
    prev_df = None
    if os.path.exists(admet_props_path):
        prev_df = pd.read_csv(admet_props_path)
        prev_df.set_index('smiles', inplace=True)

    #print(len(output_smiles))
    #print(prev_df.shape, len(prev_df.index))

    # compute plog, qed, drd2, sas for the optimized smiles and merge with previous properties
    props = defaultdict(dict)
    for smi in output_smiles:
        if prev_df is not None and smi in prev_df.index:
            for col in prev_df.columns:
                props[smi][col] = prev_df.loc[smi][col]
        elif prev_df is not None:
            continue

        if 'plogp' not in props[smi]:
            props[smi]['plogp'] = penalized_logp(smi)
        if 'qed' not in props[smi]:
            props[smi]['qed'] = qed(smi)
        if 'drd2' not in props[smi]:
            props[smi]['drd2'] = drd2(smi)
        if 'sas' not in props[smi]:
            props[smi]['sas'] = sas(smi)
    
    #print(len(props))
    # save the props as a dataframe
    props_df = pd.DataFrame(props).T
    #print(props_df.shape, len(props_df.index))
    props_df = props_df.reset_index().rename(columns={'index': 'smiles',
                                                      'BBBP': 'bbbp', 'AMES': 'mutagenicity', 
                                                      'HIA_Hou': 'hia'})
    props_df = props_df.round(2)
    #print(props_df.shape, len(props_df.index))
    props_df.to_csv(props_path, index=False)

### Evaluation metrics

In [10]:
def compute_metrics(test_data_path, output_path, 
                    seen_smiles, task,
                    props_path,
                    normalize = {'plogp': (-20, 10)}):
    """
    Compute the metrics for the optimization task.
    Args:
        test_data_path (str): Path to the test data json file.
        output_path (str): Path to the output file.
        seen_smiles (set): Set of SMILES seen during training.
        property (list): List of properties to optimize.
        task (str): Name of the task.
        props_path (str): Path to the properties file.
        normalize (dict): Dictionary of properties to normalize.
    """
    # SR is the fraction of molecules that have properties either decreased or increased as per the prompt
    # measure novelty as the fraction of valid and successful molecules that are not in the training set
    # measure similarity with input smiles as the average Tanimoto similarity between the input and the successful optimized molecules
    # measure diversity as the average pairwise Tanimoto similarity among the successful optimized molecules
    # measure synthetic accessibility as the average SAS score of the successful optimized molecules
    # measure change percentage as the average percentage change in the properties of the successful and valid molecules

    test_data = load_json(test_data_path)
    test_data = [data for data in test_data if data['task'] == task and data['instr_setting'] == 'seen']
    print(len(test_data))

    property = task.split('+')
    input_props = []
    input_smiles = []

    for i in range(len(test_data)):
        input_smiles.append(test_data[i]['source_smiles'])
        input_prop = {}        
        for prop in property:
            input_prop[prop] = test_data[i]['properties'][prop]['source']
        input_props.append(input_prop)

    output_smiles = []
    with open(output_path, 'r') as f:
        for line in f:
            output_smiles.append(line.strip())
    
    #output_smiles = output_smiles[:len(test_data)]
    if not os.path.exists(props_path):
        raise ValueError(f"Properties file {props_path} does not exist. Run compute_props() to generate the properties file.")

    props_df = pd.read_csv(props_path)
    props_df.set_index('smiles', inplace=True)

    SR = diversity = 0
    SAS = []
    num_invalid = 0
    num_seen_smiles = 0
    most_optimized_smiles = []
    perc_change = defaultdict(list)
    norm_perc_change = defaultdict(list)
    avg_change = defaultdict(list)
    avg_value = defaultdict(list)
    composite_change = []
    norm_composite_change = []
    similarity_with_input = []
    num_inputs = len(test_data)

    # save the most optimized smiles and their properties
    most_optimized_smiles_props = []
    unseen_smiles = set()

    # compute properties for all smiles for each input
    for i, opt_smiles in enumerate(output_smiles):
        # if the row is empty, there were no smiles parsed by the process-output.ipynb
        if not opt_smiles:
            num_invalid += 1
            continue

        opt_smiles = opt_smiles.split(',')
        
        # otherwise, they were parseable, but could still be invalid based on rdkit
        all_invalid_gen = True
        output_props = defaultdict(list)
        for smile in opt_smiles:
            # assume that the properties were precomputed for all output smiles
            # hence, if the smile is not in the properties dataframe, then prob it is invalid
            if props_path and smile not in props_df.index:
                if smile != 'Invalid':
                    #print(i, smile)
                    unseen_smiles.add(smile)
                continue
            # if there was any one valid generation out of at most 20 opt_smiles generated
            all_invalid_gen = False
            for prop in property:
                output_props[prop].append(props_df.loc[smile, prop])
            output_props['sim'].append(pair_similarity(input_smiles[i], smile, "binary"))
        
        num_invalid += 1 if all_invalid_gen else 0
        # find the most optimized smiles based on the most improvement that satisfies the constraints
        best_idx = find_best_optimized(input_props[i], output_props, property)
        if best_idx is None:
            continue
    
        all_props_desired = True
        # increase the success count if the best optimized smile has better properties
        for j, prop in enumerate(property):
            #change = test_data[i]['properties'][prop]['change']
            change = -1 if prop == 'mutagenicity' else 1
            if np.sign(output_props[prop][best_idx] - input_props[i][prop]) != np.sign(change):
                all_props_desired = False
                break

        # find_best_optimized should return the best optimized molecule that satisfies the constraints and
        # has the most improvement in the properties
        assert all_props_desired
        
        # if all_props_desired:
        #     print_str = f"{i}, {input_smiles[i]}"
        #     for prop in output_props.keys():
        #         print_str += f", {prop}: ({input_props[i].get(prop, 0)}, {output_props[prop][best_idx]})"
        #     print(print_str)

        if all_props_desired:
            most_optimized_smiles.append(opt_smiles[best_idx])
            most_optimized_smiles_props.append({'opt_smiles': opt_smiles[best_idx], 
                                                'source_smiles': input_smiles[i],
                                                'source_properties': input_props[i],
                                                'optimized_properties': {prop: output_props[prop][best_idx] for prop in property}})
            #SR += 1
            if opt_smiles[best_idx] in seen_smiles:
                num_seen_smiles += 1
            similarity_with_input.append(output_props['sim'][best_idx])
            try:
                SAS.append(sas(opt_smiles[best_idx]))
            except:
                pass
            for prop in property:
                avg_value[prop].append(output_props[prop][best_idx])
                input_val = input_props[i].get(prop, 0)
                output_val = output_props[prop][best_idx]
                # we can compute the absolute change in the property because
                # we already ensured that this is optimized
                avg_change[prop].append(abs(output_val - input_val))
                
                if input_val == 0:
                    perc_change[prop].append((output_val - 0.1)/0.1)
                else:
                    perc_change[prop].append(abs(input_val - output_val) / abs(input_val))

                # for percentage change, compute an unnormalized and normalized change
                if normalize and prop in normalize:
                    input_val = normalize_prop(input_val, *normalize[prop])
                    output_val = normalize_prop(output_val, *normalize[prop])
    
                    if input_val == 0:
                        norm_perc_change[prop].append((output_val - 0.1)/0.1)
                    else:
                        norm_perc_change[prop].append(abs(input_val - output_val) / abs(input_val))

            composite_change.append(np.mean([perc_change[prop][-1] for prop in property]))
            if normalize:
                norm_composite_change.append(np.mean([norm_perc_change[prop][-1] for prop in property]))
            
    fp_sim = compute_FP_sim(most_optimized_smiles)
    diversity = 1 - np.mean(fp_sim[np.triu_indices(len(fp_sim), k=1)])
    num_success = len(most_optimized_smiles)
    num_unseen = num_success - num_seen_smiles
    num_valid = num_inputs - num_invalid
    #print(len(unseen_smiles))
    
    SR = num_success / num_inputs
    SR_V = num_success / num_valid if num_valid else 0
    validity = num_valid / num_inputs

    ret = {
        "SR": SR*100,
        "Sim": np.mean(similarity_with_input),
        "RI": np.mean(composite_change),
        "Val": validity*100,
        "SR (V)": SR_V*100,
        "Nov": (num_unseen / num_success) * 100 if num_success else 0,
        "Div": diversity,
        "SAS": np.mean(SAS),
        "Norm RI": np.nanmean(norm_composite_change)
    }
    for prop in property:
        ret[f"{prop}-APS"] = np.mean(avg_value[prop])
        ret[f"{prop}-impv"] = np.mean(avg_change[prop])
        ret[f"{prop}-impv%"] = np.mean(perc_change[prop]) * 100
        if normalize:
            ret[f"{prop}-impv(n)%"] = np.mean(norm_perc_change[prop]) * 100

    return ret, most_optimized_smiles_props

### Evaluation Examples

In [14]:
os.chdir('/Users/van/Developer/workspace/gellmo/GeLLMO')  # Go back to the root directory of the project
seen_smiles = np.loadtxt(f"data/unique_mols_in_one_ds_pairs.txt", dtype=str, comments=None)
seen_smiles = set(seen_smiles)

task = 'bbbp+drd2+qed'
test_data_path = 'test.json'
output_path = f"examples/{task}-smiles.csv"
admet_props_path = f"examples/{task}-admet_props.csv"
props_path = f"examples/{task}-props.csv"
# generate the properties for the optimized smiles
compute_props(output_path, admet_props_path, props_path)
# compute the metrics for the optimized smiles
ret, _ = compute_metrics(test_data_path, output_path, seen_smiles, task, props_path, None)
# print the metrics converting to 2 decimal places
print(pd.DataFrame(ret, index=[0]).T.round(2))

500
                  0
SR            85.80
Sim            0.59
RI             4.78
Val           99.60
SR (V)        86.14
Nov          100.00
Div            0.84
SAS            2.93
Norm RI         nan
bbbp-APS       0.75
bbbp-impv      0.37
bbbp-impv%   140.87
drd2-APS       0.19
drd2-impv      0.18
drd2-impv% 1,188.92
qed-APS        0.40
qed-impv       0.19
qed-impv%    104.15


/opt/homebrew/Caskroom/miniforge/base/envs/sk21/lib/python3.6/site-packages/ipykernel_launcher.py:180: RuntimeWarning: Mean of empty slice
